# DSGVO-konforme Anonymisierung mit k-Anonymität, l-Diversität und t-Closeness
Dieses Notebook zeigt, wie man einen Datensatz DSGVO-konform anonymisiert:
- Entfernen direkter Identifikatoren
- Generalisierung von Quasi-Identifikatoren
- Prüfung auf k-Anonymität, l-Diversität und t-Closeness


In [4]:
import pandas as pd
from scipy.spatial.distance import euclidean

# 1. Daten laden
df = pd.read_csv(r"C:\Users\DMQDCQR\Downloads\hallo\new_retail_data_reduced.csv")

print("=== Originaldaten ===")
print(df.head(), "\n")

# 2. Quasi-Identifikatoren generalisieren
df['Zipcode'] = df['Zipcode'].astype(str).str[:2]  # PLZ auf 2 Stellen
df['Age'] = pd.cut(df['Age'], bins=[0, 30, 50, 100], labels=['<=30', '31-50', '51+'])

# 3. k-Anonymität prüfen
k = 2
quasi_identifiers = ['Zipcode', 'Age']
group_sizes = df.groupby(quasi_identifiers).size().reset_index(name='Count')
df_k_anonym = df.merge(group_sizes[group_sizes['Count'] >= k], on=quasi_identifiers)

print(f"Nach k-Anonymität (k={k}): {len(df_k_anonym)} von {len(df)} Datensätzen\n")

# 4. l-Diversität prüfen
l = 2
sensitive_attr = 'Income'

def check_l_diversity(group):
    return group[sensitive_attr].nunique() >= l

df_l_diverse = df_k_anonym.groupby(quasi_identifiers).filter(check_l_diversity)
print(f"Nach l-Diversität (l={l}): {len(df_l_diverse)} von {len(df_k_anonym)} Datensätzen\n")

# 5. t-Closeness prüfen
t = 0.3
overall_dist = df[sensitive_attr].value_counts(normalize=True)

def t_closeness(group):
    group_dist = group[sensitive_attr].value_counts(normalize=True)
    group_dist = group_dist.reindex(overall_dist.index, fill_value=0)
    return euclidean(overall_dist.values, group_dist.values) <= t

df_t_close = df_l_diverse.groupby(quasi_identifiers).filter(t_closeness)
print(f"Nach t-Closeness (t={t}): {len(df_t_close)} von {len(df_l_diverse)} Datensätzen\n")

# 6. Ergebnis anzeigen
print("=== Anonymisierte Daten (ersten 10 Zeilen) ===")
print(df_t_close.head(10))


=== Originaldaten ===
   Zipcode   Age    Country Income
0  77985.0  21.0    Germany    Low
1  99071.0  19.0         UK    Low
2  75929.0  48.0  Australia    Low
3  88420.0  56.0     Canada   High
4  48704.0  22.0         UK    Low 



C:\Users\DMQDCQR\AppData\Local\Temp\1\ipykernel_15632\2258582475.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_sizes = df.groupby(quasi_identifiers).size().reset_index(name='Count')
C:\Users\DMQDCQR\AppData\Local\Temp\1\ipykernel_15632\2258582475.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_l_diverse = df_k_anonym.groupby(quasi_identifiers).filter(check_l_diversity)


Nach k-Anonymität (k=2): 301837 von 302010 Datensätzen

Nach l-Diversität (l=2): 301837 von 301837 Datensätzen



C:\Users\DMQDCQR\AppData\Local\Temp\1\ipykernel_15632\2258582475.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_t_close = df_l_diverse.groupby(quasi_identifiers).filter(t_closeness)


Nach t-Closeness (t=0.3): 301837 von 301837 Datensätzen

=== Anonymisierte Daten (ersten 10 Zeilen) ===
  Zipcode   Age  Country  Income  Count
0      77  <=30  Germany     Low   1423
1      77  <=30       UK    High   1423
2      77  <=30       UK    High   1423
3      77  <=30       UK    High   1423
4      77  <=30       UK    High   1423
5      77  <=30       UK    High   1423
6      77  <=30       UK    High   1423
7      77  <=30       UK    High   1423
8      77  <=30       UK  Medium   1423
9      77  <=30       UK  Medium   1423


„Nach Anwendung der Generalisierung und Entfernung direkter Identifikatoren wurde der Datensatz so reduziert, dass er die Anforderungen an k-Anonymität (k=2), l-Diversität (l=2) und t-Closeness (t=0.3) vollständig erfüllt. 301 837 Datensätze verblieben anonymisiert, was 99,94 % des Originaldatensatzes entspricht.“